In [ ]:
import networkx as nx
import numpy as np
from scipy.optimize import linprog
from scipy.stats import norm
from copy import deepcopy

In [ ]:
import os
import logging
import numpy as np
from datetime import datetime

## Global Default Weight Variable
This is used to try out different default variables for edges that are supposed to be trivial to traverse.

## Pre-Processing

Please note this difference:
Adding virtual start node: We ADD a new virtual starting node and connect it to existing root (starting nodes)
Adding virtual target node: we REMOVE the old target nodes and replace them with one joint virtual target node

### Add Virtual starting node

In [ ]:
def find_and_add_entry_node(graph):
    # First identify the original root nodes
    original_roots = [n for n, deg in graph.in_degree() if deg == 0]
    
    if len(original_roots) > 1:
        # add virtual entry node
        entry = 0  # virtual entry node
        graph.add_node(entry)
        for r in original_roots:
            graph.add_edge(entry, r, weight=DEFAULT_WEIGHT_VALUE)
        return entry, graph, original_roots
    else:
        # Only one root, use it as entry
        entry = original_roots[0]
        return entry, graph, original_roots

### Add Virtual Target Node 

In [ ]:
def merge_targets_with_multi_edges(orig_graph):
    ## Part 1: This part is concerned only with creating a list of target nodes
    targets = []
    
    # Iterate through all nodes and their out degrees
    for node, out_degree in orig_graph.out_degree():
        # If node has no outgoing edges (degree=0), it's a target
        if out_degree == 0:
            targets.append(node)
    
    ## Part 2: Simple check that just returns the original graph if there are no targets
    if len(targets) <= 1:
        return orig_graph

    # Create just a merged label for the new virtual target node
    merged_label = "c(" + ",".join(str(t) for t in targets) + ")"


    ## Create new MultiDiGraph without edge weights for now
    
    ## Part 3: This part creates a new graph that replaces the original target nodes 
    # with the new virtual target node
    # Immportant: This part does NOT yet add the specific edge weights between the nodes

    # create list that contains all the non-target nodes
    newG = nx.MultiDiGraph()

    # create list that contains all the non-target nodes
    non_targets = []
    for node in orig_graph.nodes():
        if node not in targets:
            non_targets.append(node)
            
    # Add all non-target nodes to new graph
    for node in non_targets:
        newG.add_node(node)
        
    # Add the virtual target node
    newG.add_node(merged_label)
    
    ## Part 4: Edge Recreation between source nodes and the new Virtual Target Node
    # This entire section ensures we maintain all parallel edges and their weights just like R does

    # Track edges 
    pred_target_edges = {}

    ## Part 4a: Collect ALL Edges going to Original Target Nodes

    # We need this info to recreate these edges later with the virtual target node
    # Example of what we're building:
    # If node 5 has these edges:
    # - Edge to node 15 with weight 0.3
    # - Another edge to node 15 with weight 0.7
    # - Edge to node 16 with weight 0.3
    # Then pred_target_edges[5] will contain: [(0.3, '15'), (0.7, '15'), (0.3, '16')]
    for u, v, data in orig_graph.edges(data=True):
        if v in targets:
            if u not in pred_target_edges:
                pred_target_edges[u] = []
            weight = data.get('weight', DEFAULT_WEIGHT_VALUE)
            pred_target_edges[u].append((weight, v))

    ## Part 4b: Count How Many Times Each Weight Appears for Each Source Node

    # For each source node, we count duplicate weights
    # Example: If node 5 has three edges with weights [0.3, 0.7, 0.3]
    # Then weight_counts will be {0.3: 2, 0.7: 1}
    for u, edges in pred_target_edges.items():
        weight_counts = {}
        for weight, _ in edges:
            weight_counts[weight] = weight_counts.get(weight, 0) + 1

        ## Part 4c: For this one source node, finally, create the actual edges to our virtual target
        
        # If weight_counts shows {0.3: 2, 0.7: 1}, we create:
        # - 2 parallel edges with weight 0.3
        # - 1 edge with weight 0.7
        for weight, count in weight_counts.items():
            for _ in range(count):
                newG.add_edge(u, merged_label, weight=weight)
    
    ## Part 5: Copy Over All Other Edges That Don't Touch Target Nodes

    # OK now for the easy part - just copy over all other edges 
    # Example: if we have edge from node 1 -> node 2 with weight 0.5 
    # AND neither node 1 or 2 are target nodes, we just copy it exactly as is

    for u, v, data in orig_graph.edges(data=True):
        # Skip any edges that touch target nodes - we already dealt with those in part 4
        if v not in targets and u not in targets:
            # **data is used to unpacks all attributes automatically
            # So if our edge had data = {'weight': 0.5, 'color': 'red'}
            # This line becomes: newG.add_edge(u, v, weight=0.5, color='red')
            newG.add_edge(u, v, **data)

    return newG

## Elements preparation for the Game

In [ ]:
def generate_game_elements(graph, entry_node, original_roots):
    """
    This function sets up all the elements needed for our game after we've preprocessed our graph.
    IMPORTANT: This function only determines the POSSIBLE PATHS through the graph.
    The actual weights and parallel edges are used later in calculate_payoff_distribution!

    Returns:
    routes: List of all possible paths from entry to target (ignoring parallel edges)
    V: List of all nodes that appear in any route
    as1: List of nodes where defender can place defenses (excludes entry, target, root nodes)
    as2: Same as routes - all possible attack paths 
    target_list: Single-item list containing our virtual target node
    node_order: Nodes in V but sorted in topological order (helps with game calculations)
    adv_list: List of nodes where attacker could be (excludes entry and target nodes)
    theta: Dictionary mapping each possible attacker location to its probability (all equal)
    m: Number of attack paths
    """
    ## Part 1: Find our Target Node - Should be Only One!
    target_list = [n for n,d in graph.out_degree() if d == 0]
    if len(target_list) != 1:
        print("WARNING: Expected exactly one target node after contraction. Found:", target_list)
    
    ## Part 2: Get all UNIQUE Possible Attack Routes Through Graph
    # This means finding all possible node sequences like [0->1->2->target]
    # Note: At this stage we don't care about edge weights - those come into play later!

    ## Part 2a: Get Initial Raw Path List, including duplicates from parallel edges
    # Example: If we have two edges between 1->2, we might get:
    #   Path1: [0->1->2->target] (using first 1->2 edge)
    #   Path2: [0->1->2->target] (using second 1->2 edge)
    raw_routes = list(nx.all_simple_paths(graph, entry_node, target_list[0]))

    ## Part 2b: Remove Duplicate Paths
    # Example: The two paths above would consolidate to just:
    #   [0->1->2->target]
    # Why? Because for now we only care about WHICH nodes can be visited,
    # not HOW (i.e., which specific edges with which weights)
    consolidated_routes = []
    seen_paths = set()

    for path in raw_routes:
        path_key = tuple(path)
        if path_key not in seen_paths:
            seen_paths.add(path_key)
            consolidated_routes.append(list(path))

    # routes now contains our final list of unique possible attack paths
    routes = consolidated_routes

    ## Part 3: Create Node Sets We Need for the Game
    # Part 3a: Get all unique nodes (V) that appear in any route
    V = sorted(set(node for path in routes for node in path), key=str)

    # Part 3b: Get nodes in proper order (helps with game calculations later)
    topo_all = list(nx.topological_sort(graph))
    node_order = []
    for n in topo_all:
        if n in V:
            node_order.append(n)
    
    ## Part 4: Create Special Node Lists for Game Logic

    # Part 4a: Create as1 which is a list of potential defender check locations
    # We exclude:
    # - The entry node (obvious - it's just virtual)
    # - The target node (game is over if attacker reaches it)
    # - Original root nodes (they're no longer part of the game)
    excluded = {entry_node} | set(target_list) | set(original_roots)

    # as1 is our list of nodes where the defender can check
    as1 = []
    for n in V:
        if n not in excluded:
            as1.append(n)

    # Part 4b: Set up attack paths (as2)
    # These are just our consolidated routes - we don't care here about the weights
    as2 = routes

    ## Part 5: Set up Game Parameters
    # Part 5a: Create list of possible attacker locations
    # Similar to as1 but we CAN use original roots - attacker might start there!
    adv_list = []
    excluded_nodes = set([entry_node]) | set(target_list)
    for n in V:
        if n not in excluded_nodes:
            adv_list.append(n)
    
    if len(adv_list) == 0:
        print("WARNING: No adversary intermediate locations found. Check graph structure.")

    # Part 5b: Calculate initial probabilities for each possible attacker location
    theta = {loc: 1/len(adv_list) for loc in adv_list} if adv_list else {}
    
    # Part 5c: Count total number of attack paths
    m = len(routes)
    
    return routes, V, as1, as2, target_list, node_order, adv_list, theta, m

## Calculate Payoffs based on the game elements

##### Method necessary to calculate Pay offs within

This method is mostly useless and can be removed long term. In my current implementation all we care about is the last entry of U = [0.17, 0.10, 0.63, 0.10, 1e-7] which represents the prob. of the attacker to reach the target node.

And per one U (Node Check & Attack Path pair), we extract only this one value for the final pay off matrix. 
lossDistribution() does not add anything of value.

In [ ]:
# Note: The current version is completely useless. What we later need is just the last entry of U
# for which we do not need this entire method. I just kept it because it might be necessary to add
# complexity later on.
def lossDistribution(U):
    """
    Creates standardized format matching R's lossDistribution output.
    U is already normalized and has no zeros due to preprocessing.
    Only U[-1] (last entry of dpdf) is actually used in solve_game.
    """
    return {
        'dpdf': U,  # Already normalized and cleaned in calculate_payoff_distribution
        'support': np.arange(1, len(U) + 1),
        'cdf': np.cumsum(U),
        'tail': 1 - np.cumsum(U) + U,
        'range': [1, len(U)]
    }

#### End of Explanations

#### Calculate Payoffs for each path/check pair

In [ ]:
def calculate_payoff_distribution(graph, as1, as2, V, adv_list, theta, random_steps_fn, 
                                 attack_rate, defense_rate, node_order):
   """
   This function calculates probability distributions for where attackers might end up in the graph
   for each defender-attacker strategy combination.
   
   For each check location & attack path pair:
   1. Creates vector U to store final probabilities for each node
   2. For each possible attacker starting position (avatar):
      - If on attack path: Calculates probabilities of reaching each node up to defender's check
      - If not on path: Stays at current position with probability 1.0
   3. Weights and combines all starting position probabilities
   4. Returns smoothed probability distribution

   The edge weights in the graph affect movement probabilities calculated by random_steps_fn.

   Returns:
   payoffs: List of probability distributions, one for each check location + attack path pair
   """
   payoffs = []

   ## Part 1: Process Each Defender Check + Attack Path Combination

   # For each pair, we calculate:
   # 1. Where attackers starting from different positions might end up
   # 2. How the defender's check point affects these probabilities
   # 3. A final combined probability distribution across all nodes U
   for check in as1:
       for path in as2:
           U = np.zeros(len(V))

           # print(f"\n++++++++++++++++++++++++++++++++")
           # print(f"attack_rate = {attack_rate}, defense_rate = {defense_rate}")
           # print(f"--- Starting payoff calc for check = {check}, path = {path} ---\n")

           ## Part 2: Handle Each Possible Attacker Starting Position (avatar)

           # 1. Create temporary vector L to store probabilities for this starting position
           # 2. Calculate probabilities differently if avatar is on/off attack path
           # 3. Weight by probability of attacker starting at this position (theta)
           # 4. Add weighted probabilities to final vector U
           for avatar in adv_list:
               L = np.zeros(len(V))

               ## Part 2a: Calculate Probabilities When Attacker Starts On Path

               # If avatar is on path:
               # 1. Get remaining path from avatar's position
               # 2. Calculate movement probabilities using edge weights
               # 3. Truncate at defender's check point if present
               if avatar in path:
                   # Extract relevant portion of path from avatar position
                   start_idx = path.index(avatar)
                   route = path[start_idx:]
                   #print(f"\nProcessing avatar {avatar}:")
                   #print(f"Route from avatar: {route}")
                   
                   # Get raw movement probabilities
                   pdf_d = random_steps_fn(route, attack_rate, defense_rate, graph)
                   #print(f"PDF for entire route: {pdf_d}")

                   ## Part 2b: Adjust Probabilities Based on Defender's Check Point
                   # If defender checks on this route:
                   # 1. Truncate probabilities at check point (attacker can't go further)
                   # 2. Renormalize remaining probabilities to sum to 1
                   if check in route:
                       check_idx = route.index(check)
                       # Add 1 to include the check point itself, but don't exceed path length
                       cutPoint = check_idx + 1
                   else:
                       cutPoint = len(route)
                   #print(f"Cut point: {cutPoint}")

                   # Take probabilities up to check point and renormalize
                   # Case 1: If probabilities are effectively zero
                   # This means attacker is guaranteed to end at last reachable position
                   pdf_subset = pdf_d[:cutPoint]
                   if np.sum(pdf_subset) < 1e-15:
                       payoffDistr = np.zeros(cutPoint)
                       payoffDistr[-1] = 1.0
                   # Case 2: Normal case with non-zero probabilities
                   #   - Normalize probabilities to sum to 1.0
                   #   - Example: if pdf_subset=[0.2, 0.3] -> payoffDistr=[0.4, 0.6]
                   else:
                       payoffDistr = pdf_subset / np.sum(pdf_subset)
                   
                   #print(f"PDF subset: {pdf_subset}")
                   #print(f"Payoff distribution: {payoffDistr}")

                   # Map probabilities from path positions to node indices in V
                   route_subset = route[:cutPoint]
                   #print(f"Route subset: {route_subset}")
                   # for idx_node, node in enumerate(route_subset):
                   #     L[V.index(node)] = pdf_d[idx_node]

                   for idx_node, node in enumerate(route_subset):
                       L[V.index(node)] = payoffDistr[idx_node]
                   
                   # print("L distribution for this avatar (BEFORE weighting by Theta):")
                   # for idx_l, val in enumerate(L):
                   #     if val > 1e-10:
                   #         print(f"  Node {V[idx_l]} : {val}")

               ## Part 2c: Handle Case Where Attacker Starts Off Path
               # If avatar not on path:
               # Set probability 1.0 for staying at current position
               else:
                   L[V.index(avatar)] = 1.0
                   # print(f"\nProcessing avatar {avatar} (not in path):")
                   # print(f"L[{avatar}] = 1.0")

               ## Part 2d: Add Weighted Contribution to Final Probabilities
               
               # Weight this starting position's probabilities by theta
               # Add to running total in U
               #print(f"\nTheta[{avatar}] = {theta[avatar]}")
               U += theta[avatar] * L
               #print("Current U after adding this avatar's contribution:")
               # for idx_u, val in enumerate(U):
               #     if val > 1e-10:
               #         print(f"  Node {V[idx_u]} : {val}")


           # Ensure probabilities sum to 1 and handle edge cases
           # print(f"\n--- Aggregated U for check={check}, path={path} (BEFORE normalization) ---")
           # for idx_u, val in enumerate(U):
           #     if val > 1e-10:
           #         print(f"  Node {V[idx_u]} : {val}")
           
           ## Part 3: Clean Up and Normalize Final Distribution

           # Part 3a: Normalize and Handle Edge Cases 
           # Ensure no zero probabilities and normalize to sum to 1
           U_sum = np.sum(U)
           if U_sum < 1e-15:
               U = np.full_like(U, 1e-7)
           else:
               # normalize and prevent 0 probabilities
               U = U/U_sum
               U = np.where(U < 1e-7, 1e-7, U)
           
           # Part 3b: Reorder According to Topological Sort
           # Ensure nodes are in correct order for game logic
           node_positions = [V.index(n) for n in node_order]
           U = U[node_positions]

           # print(f"\n--- Normalized U for check={check}, path={path} ---")
           # for idx_u2, val2 in enumerate(U):
           #     if val2 > 1e-10:
           #         print(f"  Node {node_order[idx_u2]} : {val2}")

           # ld = {
           #     'dpdf': U,
           #     'support': range(1, len(U) + 1),
           #     'cdf': np.cumsum(U),
           #     'tail': 1 - np.cumsum(U) + U
           # }

           ## Part 4: Create Smoothed Distribution
           # Convert final probabilities into loss distribution object
           # print(f"Pre-lossDistribution U for check={check}, path={path}:", U)
           ld = lossDistribution(U)
           payoffs.append(ld)

           # print(f"The FINAL entry used in the pay off matrix (! this is what matters) is: {ld['dpdf'][-1]}")

   return payoffs

## Find the Equilibrium

### Method for finding optimal solutions

#### Some Explanations

This method solves a zero-sum game using linear programming. It first creates a proper payoff matrix from the loss distributions, then solves two LPs: one for defender minimizing attacker success probability, and one for attacker maximizing it. Both optimizations should yield the same equilibrium value.

- Constructs payoff matrix
- Solves LP for defender FIRST (minimizing attacker success)
- Then solves LP for attacker (maximizing their success)
- Verifies both values match (equilibrium property)

The method returns a dictionary: 

{

    'optimal_defense': {3: 0.6, 4: 0.4},  # defend node 3 with 60% probability, node 4 with 40%                     
    'attacker_strategy': [0.3, 0.5, 0.2], # use path 1 with 30% probability, path 2 with 50%, etc.
    'defender_success': 0.128,           # defender can keep attacker success ≤ 0.128              
    'attacker_success': 0.128            # attacker can achieve at least 0.128
                                            
}

#### End of Explanations

In [ ]:
def solve_game(payoffs, as1, as2):
    n = len(as1)
    m = len(as2)

    # # Add debug statements here
    # print("\n=== Debug: Strategy Mappings ===")
    # print("Defender strategies (as1):", as1)
    # print("Attacker paths (as2):")
    # for idx, path in enumerate(as2):
    #     print(f"Path {idx}:", path)
    
    # Create payoff matrix
    payoff_matrix = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            idx = i*m + j
            ld = payoffs[idx]
            payoff_matrix[i, j] = ld['dpdf'][-1]
            #print(f"Which finally leads to the following entry in the payoff matrix: {payoff_matrix[i, j]}")


    # NEW DEBUG CODE: Print payoff matrix before optimization
    if debug_mode:
        print("\n=== Debug: Final Payoff Matrix ===")
        print(f"Matrix dimensions: {n} x {m}\n")
        print("Payoff Matrix (probability of reaching target):")
        for i in range(n):
            row_str = f"Row {i+1:2d}:"
            for j in range(m):
                row_str += f" {payoff_matrix[i,j]:8.6f}"
            print(row_str)
        print("\n=== End Debug: Final Payoff Matrix ===\n")
    
    ### Start Defender's optimization ###
    c = np.zeros(n+1)
    c[0] = 1.0
    
    A_ub = np.zeros((m, n+1))
    b_ub = np.zeros(m)
    for j in range(m):
        A_ub[j,0] = -1.0
        for i in range(n):
            A_ub[j,i+1] = payoff_matrix[i,j]
            
    A_eq = np.zeros((1, n+1))
    A_eq[0,1:] = 1.0
    b_eq = np.array([1.0])
    
    bounds = [(0,None)]*(n+1)
    
    v_defender = None
    v_attacker = None
    
    # Solve the LP for defender's strategy
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    
    ### End Defender's optimization ###

    if res.success:
        # Extract the results for later logging
        v_defender = res.x[0]
        x_def = res.x[1:]
        
        ### Start Attacker's optimization ###
        c_att = np.zeros(m+1)
        c_att[0] = -1.0
        
        A_ub_att = np.zeros((n, m+1))
        b_ub_att = np.zeros(n)
        for i in range(n):
            A_ub_att[i,0] = 1.0
            for j in range(m):
                A_ub_att[i,j+1] = -payoff_matrix[i,j]
                
        A_eq_att = np.zeros((1, m+1))
        A_eq_att[0,1:] = 1.0
        b_eq_att = np.array([1.0])
        
        bounds_att = [(0,None)]*(m+1)

        # solve the LP for attacker's strategy
        res_att = linprog(c_att, A_ub=A_ub_att, b_ub=b_ub_att, 
                         A_eq=A_eq_att, b_eq=b_eq_att, bounds=bounds_att)
        
        ### End Attacker's optimization ###
        
        if res_att.success:
            # Extract attacker results for later logging
            y_att = res_att.x[1:]
            v_attacker = res_att.x[0]   # new - remove the negative sign because c_att[0] = -1.0
            
            # Now both values are defined, we can check
            if abs(v_defender - v_attacker) > 1e-5:
                logger.info("\nWarning: Defender and attacker values don't match!")
                logger.info(f"Defender value: {v_defender:.6f}")
                logger.info(f"Attacker value: {v_attacker:.6f}")
            
            return {
                'optimal_defense': dict(zip(as1, x_def)),
                'attacker_strategy': y_att,
                'defender_success': v_defender,
                'attacker_success': v_attacker
            }
    
    logger.info("LP optimization failed")
    return None

### Debug Code (Can be deleted sooner or later)

In [ ]:
def print_debug_info(graph, stage=""):
    print(f"\n{stage}:")
    print("Nodes:", list(graph.nodes()))
    print("Total list of Edges with their weights:")
    if isinstance(graph, nx.MultiDiGraph):
        # For MultiDiGraph, we need to handle multiple edges between same nodes
        for u, v, key, data in graph.edges(data=True, keys=True):
            weight = data.get('weight', DEFAULT_WEIGHT_VALUE)  # default to 1 if no weight
            print(f"{u} -> {v} (key={key}) : {weight}")
    else:
        # Original printing for regular DiGraph
        for u, v, data in graph.edges(data=True):
            weight = data.get('weight', DEFAULT_WEIGHT_VALUE)  # default to 1 if no weight
            print(f"{u} -> {v} : {weight}")

#### Brute force re-ordering method for Experiment 3 & 4 for easier debugging. 
Useless for other Experiments

In [ ]:
def reorder_strategies(as1, as2):
    # R's order: [5, 15, 6, 8, 10, 7, 11, 9]
    r_order = [5, 15, 6, 8, 10, 7, 11, 9]
    
    # Print desired order information
    print("\nDebug - Reordering process:")
    print("Desired order:", r_order)
    
    # Create new ordered list
    as1_reordered = []
    for node in r_order:
        if node in as1:
            as1_reordered.append(node)
    
    # Reorder paths according to R's path ordering
    r_paths = [
        [0, 1, 5, 15, 'c(12,13,14,16)'],
        [0, 3, 6, 8, 10, 15, 'c(12,13,14,16)'],
        [0, 3, 6, 8, 'c(12,13,14,16)'],
        [0, 3, 8, 10, 15, 'c(12,13,14,16)'],
        [0, 3, 8, 'c(12,13,14,16)'],
        [0, 4, 7, 10, 15, 'c(12,13,14,16)'],
        [0, 4, 7, 'c(12,13,14,16)'],
        [0, 2, 11, 'c(12,13,14,16)'],
        [0, 2, 9, 'c(12,13,14,16)'],
        [0, 2, 10, 15, 'c(12,13,14,16)'],
        [0, 2, 'c(12,13,14,16)']
    ]
    
    # Create new ordered list of paths
    as2_reordered = []
    for r_path in r_paths:
        for path in as2:
            if path == r_path:
                as2_reordered.append(path)
                break
    
    return as1_reordered, as2_reordered

def debug_paths(as1, as2):
    # Print debug information for strategies
    print("\nDebug - Current defender Node Check locations:")
    print("as1:", as1)
    
    # Print debug information for paths
    print("\nDebug - Current Attack paths:")
    for i, path in enumerate(as2):
        print(f"  {i}: {path}")
    
    return

### Method to run the Actual Game

In [ ]:
def run_game(attacker_graph, defender_graph=None, attack_rate_list=None, dropped=None, defense_rate_list=None, random_steps_fn=None):
    # For backward compatibility and initial testing
    if defender_graph is None:
        defender_graph = attacker_graph
        
    # Use attacker_graph for all operations initially
    # graph = attacker_graph
    final_eq = None

    ####### Graph pre processing for ATTACKER #######
    if debug_mode:
        print_debug_info(attacker_graph, "This is the Attacker Graph")

    atk_virtual_entry_node, attacker_graph, atk_original_roots = find_and_add_entry_node(attacker_graph)
    attacker_graph = merge_targets_with_multi_edges(attacker_graph) 

    if debug_mode:
        print_debug_info(attacker_graph, "After merging targets of attack graph")


    ####### Calculate ATTACKER elements #######
    _, V, _, as2, target_list, node_order, adv_list, theta, m = generate_game_elements(
        attacker_graph, atk_virtual_entry_node, atk_original_roots)



    ####### Testing pre-processing for defender_graph #######
    if debug_mode:
        print_debug_info(defender_graph, "This is the Defender Graph")
    def_virtual_entry_node, defender_graph, def_original_roots = find_and_add_entry_node(defender_graph)
    defender_graph = merge_targets_with_multi_edges(defender_graph) 

    if debug_mode:
        print_debug_info(defender_graph, "After merging targets of the Defender Graph")

    ####### Calculate DEFENDER elements #######
    _, _, as1, _, _, _, _, _, _ = generate_game_elements(defender_graph, def_virtual_entry_node, def_original_roots)

    # Forceful reordering of strategies to be better able to debug 
    # Remove later used mainly for experiment 3
    # as1, as2 = reorder_strategies(as1, as2)

    # Some debugs
    if debug_mode:
        print("\n Dropped nodes are: ", dropped)
        debug_paths(as1, as2)


    
    if not defense_rate_list:
        defense_rate_list = [0]
    if not attack_rate_list:
        attack_rate_list = [0]
    
    for defenseRate in defense_rate_list:
        for attackRate in attack_rate_list:
            logger.info("\n++++++++++++++++++++++++++++++++")
            logger.info(f"\nThe virtual target nodeID is {target_list[0]}\n")
            logger.info(f"attack rate =  {attackRate} , defense rate =  {defenseRate} \n")
            logger.info("\tequilibrium for multiobjective security game (MOSG)\n")
            

            # changing this was important because the function is now called with the graph
            payoffs = calculate_payoff_distribution(
                attacker_graph, as1, as2, V, adv_list, theta, 
                random_steps_fn,  # Just pass the function directly
                attackRate, defenseRate, node_order
            )

            # debug_payoff_matrix(payoffs, as1, as2)
            
            eq = solve_game(payoffs, as1, as2)
            if eq is not None:
                final_eq = eq  
                logger.info("optimal defense strategy:")
                logger.info("         prob.")
                for node, prob in sorted(eq['optimal_defense'].items(), key=lambda x: str(x[0])):
                    logger.info(f"{node} {prob:.6e}")
                
                logger.info("\nworst case attack strategies per goal:")
                logger.info("          1")
                if 'attacker_strategy' in eq:
                    for idx, prob in enumerate(eq['attacker_strategy'], 1):
                        logger.info(f"{idx} {prob:.7f}")
                logger.info(f"[1] {eq['attacker_success']:.3f}")
                
                logger.info(f"\nDefender can keep attacker success below: {eq['defender_success']:.3f}")
                logger.info(f"Attacker can guarantee success probability of: {eq['attacker_success']:.3f}")

    return final_eq

### Method to be called in different Notebook

In [ ]:
def main():
    # Make sure we're using the main logger
    switch_logger(use_subgraph_logger=False)
    
    # First run with full graph
    logger.info("\n\n")
    logger.info("="*80)
    logger.info("BASELINE RUN: BOTH ATTACKER AND DEFENDER HAVE FULL GRAPH KNOWLEDGE")
    logger.info("="*80)
    logger.info("\n")
    
    # Run the baseline
    print(f"We start with the baseline graph calculation!")
    baseline_result = run_game(attacker_graph=full_attack_graph, defender_graph=full_attack_graph, attack_rate_list=attack_rate_list, 
                              defense_rate_list=defense_rate_list, random_steps_fn=random_steps)
    
    # If toggle is False, also run subgraph analysis
    print(f"Now we are going to run the subgraph analysis")
    if not RUN_BASELINE_ONLY:
        # Switch to subgraph logger
        switch_logger(use_subgraph_logger=True)
        
        logger.info("\n")
        logger.info("="*80)
        logger.info(f"STARTING SUBGRAPH ANALYSIS WITH {len(defender_subgraphs_list)} DEFENDER SUBGRAPHS")
        logger.info(f"Drop percentage is : {current_drop_percentage}")
        logger.info("="*80)
        logger.info("\n")
        
        # List to store defender success values
        attacker_success_values = []
        # List to store defender success values
        attacker_success_values = []
        # New list to track total nodes dropped in each run
        total_nodes_dropped_list = []
        
        # Run the subgraph analysis
        for i in range(len(defender_subgraphs_list)):
            # Extract subgraph and dropped nodes from the tuple
            defender_subgraph, dropped_nodes, cleaned_nodes = defender_subgraphs_list[i]

            # Track total nodes dropped for this run
            total_nodes_dropped = len(dropped_nodes) + len(cleaned_nodes)
            total_nodes_dropped_list.append(total_nodes_dropped)

            logger.info("\n")
            logger.info("-"*60)
            logger.info(f"SUBGRAPH RUN #{i+1}: DEFENDER HAS LIMITED NETWORK VISIBILITY")
            logger.info(f"Drop percentage is: {current_drop_percentage}")
            logger.info(f"Node(s) Nr. {', '.join(map(str, dropped_nodes))} were dropped from this graph")
            logger.info(f"Additionally {len(cleaned_nodes)} nodes were cleaned afterwards")
            logger.info(f"So in total {total_nodes_dropped} Nodes were dropped")
            logger.info("-"*60)
            logger.info("\n")

            # Prepare the graph (deep copy only the subgraph part)
            defender_subgraph_current = deepcopy(defender_subgraph)

            result = run_game(attacker_graph=full_attack_graph, defender_graph=defender_subgraph_current, dropped = dropped_nodes,
                            attack_rate_list=attack_rate_list, 
                            defense_rate_list=defense_rate_list, random_steps_fn=random_steps)

            # Store defender success value if available
            if result and 'attacker_success' in result:
                attacker_success_values.append(result['attacker_success'])

        # Calculate and log the average
        if attacker_success_values:
            avg_attacker_success = sum(attacker_success_values) / len(attacker_success_values)
            # Calculate average total nodes dropped
            avg_total_nodes_dropped = sum(total_nodes_dropped_list) / len(total_nodes_dropped_list)

            # Get total number of nodes in the full graph
            total_nodes = full_attack_graph.number_of_nodes()

            # Calculate percentages
            initial_drop_percentage = (len(dropped_nodes) / total_nodes) * 100
            total_drop_percentage = (avg_total_nodes_dropped / total_nodes) * 100

            logger.info("\n\n")
            logger.info("="*80)
            logger.info("SUBGRAPH ANALYSIS SUMMARY")
            logger.info("="*80)
            logger.info(f"Number of subgraphs analyzed: {len(attacker_success_values)}")
            logger.info(f"Total number of nodes in the graph: {total_nodes}")
            logger.info(f"Initial number of dropped nodes per subgraph: {len(dropped_nodes)}")
            logger.info(f"% of dropped nodes (initially): {initial_drop_percentage:.1f}%")
            logger.info(f"Average total nodes dropped (including cleanup): {avg_total_nodes_dropped:.2f}")
            logger.info(f"% of total nodes dropped (including cleanup): {total_drop_percentage:.1f}%")
            logger.info(f"Baseline attacker success: {baseline_result['attacker_success']:.3f}")
            logger.info(f"Average attacker success across subgraphs: {avg_attacker_success:.3f}")
            logger.info(f"Difference from baseline: {'+' if avg_attacker_success - baseline_result['attacker_success'] > 0 else ''}{avg_attacker_success - baseline_result['attacker_success']:.3f}")
            logger.info("="*80)
            print("The subgraph analysis is done!")